In [ ]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.linear_model import LinearRegression,Ridge,ElasticNet,ElasticNetCV,LassoCV,SGDRegressor,RidgeCV
from collections import defaultdict
import json
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import os, sys

import fnmatch

In [ ]:
Xso = []
y = []

pos_Xs = defaultdict(list)
pos_y = defaultdict(list) 
pos_min = defaultdict(list) 

for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, 'ZGMH_League*.json'):
        print(file)
        data = json.load(open(file,'rt',encoding='utf-8-sig'))
        player_ratings = {}
        player_pos = {}

        valid_pos = set()
        for p in data['players']:
            for r in p['ratings']:
                player_ratings[(p['pid'],r['season'])] ={k:v for k,v in r.items() if type(v) == int and k != 'season'}
                player_pos[(p['pid'],r['season'])] =r['pos']

                valid_pos.add(r['pos'])
        for p in data['players']:
            for r in p['stats']:
                if r['min'] > 0:
                    pos = player_pos[(p['pid'],r['season'])]
                    pAV = r['ops'] + r['dps'] + r['gps']
                    if pAV:
                        pos_Xs[pos].append(player_ratings[(p['pid'],r['season'])])
                        pos_y[pos].append(float(pAV)/r['min'])
                        pos_min[pos].append(r['min'])

        valid_pos = sorted(list(valid_pos))
        valid_col = [k for k in list(player_ratings.values())[0] if k not in ['min','pos','ovr','pot','injuryIndex']]
        print(valid_pos)

        for g in data['games']:
            season = g['season']
            if g['won']['tid'] == g['teams'][0]['tid']: #home team won
                s = g['won']['pts'] - g['lost']['pts'] 
            else:
                s =  g['lost']['pts'] - g['won']['pts']
            y.append(s)
            r2 = []
            for i in range(2):
                team_r = []
                t = g['teams'][i]['players']
                for p in t:
                    pos = p['pos']
                    mp = p['min']
                    r = {k:mp*v for k,v in player_ratings[(p['pid'],season)].items()}
                    r['pos'] = pos
                    r['min'] = mp
                    if mp > 0:
                        team_r.append(r)
                # dummies to make sure we have players at every pos
                for p in valid_pos:
                    r = {k:0 for k in r}
                    r['pos'] = p
                    r['min'] = 1e-9
                    team_r.append(r)

                # should be alphabetical
                res = pd.DataFrame(team_r).groupby('pos').sum()
                res2 = res.divide(res['min'],'rows')
                ra = np.array(res2[valid_col])
                r2.append(ra)
            Xso.append(np.array(r2[0] - r2[1]).flatten())

In [ ]:
res

In [ ]:
# predict Point Shares per position
for pos in valid_pos:
    print(pos)
    #Xs = np.nan_to_num(pos_Xs[pos],0)
    fx = StandardScaler()
    y2 = np.array(pos_y[pos]).astype(float)
    X3 = sm.add_constant(pd.DataFrame(fx.fit_transform(pd.DataFrame(pos_Xs[pos])[valid_col]),columns=valid_col))
    filt = np.array(pos_min[pos]) > sorted(pos_min[pos])[len(pos_min[pos])//2]
    est = sm.OLS(y2[filt], X3[filt])
    est2 = est.fit()
    print(est2.summary())

In [ ]:
# predict Point Shares per position

for pos in valid_pos:
    #Xs = np.nan_to_num(pos_Xs[pos],0)
    fx = StandardScaler()
    y2 = np.array(pos_y[pos]).astype(float)
    X3 = pd.DataFrame(fx.fit_transform(pd.DataFrame(pos_Xs[pos])[valid_col]),columns=valid_col)
    filt = np.array(pos_min[pos]) > sorted(pos_min[pos])[len(pos_min[pos])//2]
    est = ElasticNetCV([0.001,.1,.4,.5,.7,.725,.75,.775,.8,.9,.95,.99,1],cv=3,positive=True,max_iter=3e4)
    est.fit(X3[filt],y2[filt],)
    print(pos,est.score(X3[filt],y2[filt]))
    print(pos+' : { ')
    res = sorted([(i,r) for r,i in zip(valid_col,est.coef_) if i > 0],reverse=True)
    for i,r in res:
        if i > 0:
            print('\t{}: [{:.1f}, 1],'.format(r,i/res[-1][0]))
    print('},')

In [ ]:
y.mean(),y.std()
#X3.shape,y.shape

In [ ]:
len(Xso),len(y)

In [ ]:
plt.hist(y,20)

In [ ]:
exclude = {
    ('glk'):['G'],
}
#exclude = {}
exp_lbl = sum([[str(p) + '_' + str(s) for s in valid_col] for p in valid_pos],[])
filt_lbl = []
filt_vec = []
for n in exp_lbl:
    inc = True
    pos,rate = n.split('_')
    for k,v in exclude.items():
        if pos not in v and rate in k:
            inc=False
    filt_vec.append(inc)       
    if inc:
        print(n)
        filt_lbl.append(n)

In [ ]:

Xs = np.nan_to_num(Xso,0)
fx = StandardScaler()
X2 = fx.fit_transform(Xs[:,filt_vec] )
y = np.array(y).astype(float)

reg = ElasticNetCV([.1,.4,.5,.7,.725,.75,.775,.8,.9],cv=3,n_alphas=250,positive=True,max_iter=2e4)#(alpha=0.1,l1_ratio=0.7)#CV(cv=10)#ElasticNetCV(.7,cv=10,)
#reg = ElasticNet(1e-3,0.01,positive=True,max_iter=2e4)#(alpha=0.1,l1_ratio=0.7)#CV(cv=10)#ElasticNetCV(.7,cv=10,)

#reg = lgb.LGBMRegressor()
reg.fit(X2,y)
print(X2.shape,reg.score(X2,y))

In [ ]:

plt.style.use('seaborn-white')
plt.scatter(reg.predict(X2),y,s=5,alpha=0.1)
#plt.ylim(-60,60)
#plt.xlim(-60,60)
plt.xlabel('predicted margin')
plt.ylabel('actual margin')

In [ ]:
#sorted([(i,n) for i,n in zip(reg.feature_importances_,exp_lbl) if 'LB' in n],reverse=True)

In [ ]:
X3f = pd.DataFrame(X2,columns=filt_lbl)
X3 = sm.add_constant(X3f)
est = sm.RLM(y, X3)
est2 = est.fit()
print(est2.summary())

In [ ]:
print('home field adv is {:.1f} points'.format(reg.intercept_))

In [ ]:
future_use = defaultdict(dict)
for pos in sorted(list(valid_pos)):
    res = sorted([(r,n,p) for n,r,p in zip(filt_lbl,est2.params[1:],est2.pvalues[1:]) if pos+ '_' in n],reverse=True)
    print(pos+' : { ')
    for p in res:
        if np.linalg.norm(p[0]) > 1e-3 and p[2] < 0.05:
            key = p[1].split('_')[1]
            future_use[pos][key]  = p[0]
            print('\t{}: [{:.3f}, 1],'.format(key,p[0]))
    print('},')

In [ ]:
ratings_to_use = [k for k,v in (est2.pvalues < 0.1).items() if v if k != 'const']
reg_small = ElasticNetCV(positive=True,cv=3)
reg_small.fit(Xs[:,[list(filt_lbl).index(r) for r in ratings_to_use]],y)

In [ ]:
res = sorted([(abs(v),k) for k,v in zip(ratings_to_use,reg_small.coef_)],reverse=True)
for v,k in res:
    print('{}\t{}\t{:.3f}'.format('',k,v))
print()

In [ ]:
valid_pos = set([k.split('_')[0] for k in ratings_to_use])

In [ ]:
future_use = defaultdict(dict)
for pos in sorted(list(valid_pos)):
    res = sorted([(r,n) for n,r in zip(ratings_to_use,reg_small.coef_) if pos+ '_' in n and r > 0],reverse=True)
    print(pos+' : { ')
    for p in res:
        if np.linalg.norm(p[0]) > 1e-3:
            #future_use[row[0]][res[i][1]] = row[1][res[i][1]]
            print('\t{}: [{:.3f}, 1],'.format(p[1].split('_')[1],p[0]))
    print('},')

In [ ]:
rts = defaultdict(list)
for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, 'ZGMH_League*.json'):
        print(file)
        data = json.load(open(file,'rt',encoding='utf-8-sig'))


        valid_pos = set()
        for p in data['players']:
            for r in p['ratings']:
                r2 = sum([future_use[r['pos']][k]*v for k,v in r.items() if type(v) == int and k != 'season' and k in future_use[r['pos']]])
                rts[r['pos']].append((r['ovr'],r2))

In [ ]:

i=1
for pos, results in rts.items():
    print(pos)
    plt.subplot(3,4,i)
    plt.title(pos)
    A = np.array(results)
    plt.scatter(A[:,0],A[:,1],s=5,alpha=0.5)
    plt.xlabel('current OVR')
    plt.yticks([],[])
    i+=1
plt.suptitle('Alt OVR comp',size=18,weight='bold',y=1.02)
plt.tight_layout()

In [ ]:
rts = defaultdict(list)

player_ratings = {}
player_pos = {}
player_r1 = {}
player_r2 = {}


for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, 'ZGMH_League*.json'):
        print(file)
        data = json.load(open(file,'rt',encoding='utf-8-sig'))


        valid_pos = set()
        for p in data['players']:
            for r in p['ratings']:
                r2 = sum([future_use[r['pos']][k]*v for k,v in r.items() if type(v) == int and k != 'season' and k in future_use[r['pos']]])
                
                key = (file,p['pid'],r['season'])
                player_ratings[key] ={k:v for k,v in r.items() if type(v) in [int,dict,str] and k != 'season'}
                player_pos[key] =r['pos']
                player_r1[key] = r['ovr']
                player_r2[key] = r2


                valid_pos.add(r['pos'])

In [ ]:
future_use

In [ ]:
base = json.load(open('base_hk.json','rt',encoding='utf-8-sig'))
print(len(base['players']))
for p in base['players']:
    if p['tid'] in [31,30]:
        p['tid'] = -3

print(len(base['players']))
del base['teams'][-1]['depth']
del base['teams'][-2]['depth']

max_pid = max([_['pid'] for _ in base['players']])
base['gameAttributes']['userTid'] = [{'start': None, 'value': 31}]
base['gameAttributes']['userTids'] = [31]


In [ ]:
count = {'C':5, 'D':7, 'W':10,'G':3}

min_ovr = {'C':55, 'D':55, 'W':55,'G':63}
alt_t = []

for p,N in count.items():
    mo = min_ovr[p]
    pc = {k:v for k,v in player_r1.items() if v < mo and player_pos[k] == p}
    alts = sorted([(player_r2[k],k) for k in pc],reverse=True)
    done_set = set()
    i = 0
    icnt = len(alt_t)
    while len(alt_t) < icnt + N:
        _, k = alts[i]
        if k not in done_set:
            player = {'ratings':[player_ratings[k]],'pos':p}
            alt_t.append(player)
            done_set.add(k)
        i+=1
for p in alt_t:
    p['born'] =  {'year': 1994, 'loc': 'USA'}
    p['pid'] = max_pid
    p['tid'] = 31
    p['ovrs'] = {}
    p['pots'] = {}
    #p['ratings'][0]['pos'] = p['pos']
    #del p['ratings'][0]['ovr']
    #del p['ratings'][0]['pot']
    #del p['ratings'][0]['pos']
    #del p['pos']

    max_pid += 1
    base['players'].append(p)

In [ ]:
min_ovr = {'C':63, 'D':63, 'W':63,'G':63}


alt_t = []

for p,N in count.items():
    mo = min_ovr[p]
    pc = {k:v for k,v in player_r1.items() if v > mo and player_pos[k] == p}
    alts = sorted([(player_r2[k],k) for k in pc])#,reverse=True)
    done_set = set()
    i = 0
    icnt = len(alt_t)
    while len(alt_t) < icnt + N:
        _, k = alts[i]
        if k not in done_set:
            player = {'ratings':[player_ratings[k]],'pos':p}
            alt_t.append(player)
            done_set.add(k)
        i+=1
for p in alt_t:
    p['born'] =  {'year': 1994, 'loc': 'USA'}
    p['pid'] = max_pid
    p['tid'] = 30
    p['ovrs'] = {}
    p['pots'] = {}
    #p['ratings'][0]['pos'] = p['pos']
    #del p['ratings'][0]['ovr']
    #del p['ratings'][0]['pot']
    #del p['pos']

    max_pid += 1
    base['players'].append(p)

In [ ]:
with open('overunder_h.json','wt') as fp:
    json.dump(base,fp)

In [ ]:
p